In [204]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import re
import nltk

In [205]:
file_path = 'bitchy_man.csv'

In [206]:
data = pd.read_csv('bitchy_man.csv')
text_column = data['text'].fillna('')

In [207]:
data.to_csv('bitchy_man.csv', encoding='utf-8')

In [208]:
from nltk import WordPunctTokenizer
tokenizer = WordPunctTokenizer()


text_column = text_column.apply(lambda x: ' '.join(tokenizer.tokenize(str(x).lower())))

In [209]:
text = []
for line in text_column:
  text.append(line)

text = ' '.join(text)

In [211]:
text = re.sub(r'\d+', '', text)

'правила жизни джеймса бонда :  . платит женщина . всегда .  . твои проблемы — это ее проблемы .  . твоя истерика — тоже ее проблема .  . она должна уметь драться за тебя .  . если красота и спасет мир , то вся надежда только на меня . нарисуем твою жену на холсте . ты нам даешь ее фото - мы рисуем ! закажи своей любимой женщине роскошный портрет , чтобы она увидела , какой красивой ты её видишь ! сегодня дарим  картину к заказу , чтобы порадовать еще и маму , сестру или дочку ! чтобы порадовать любимых , не нужен повод . переходи , что бы рассчитать стоимость за  минуту ! https :// vk . me / mnogokartinrf ? ref = vk сильный мужчина никогда не поднимет руку на женщину . он ударит по её кошельку ! арнольд шварцнеггер ( с ). девушки , если вы оказались в черном списке у вашего парня — еще не все потеряно . при переводе денег на его карту можно оставить сообщение !  мужчина молодеет до двадцати семи . потом он чертовски хорошеет ! дуэйн джонсон базу говорит вообще - то личный кaнал админа

In [212]:
text = re.sub(r'[:;,]', '', text)

'правила жизни джеймса бонда   . платит женщина . всегда .  . твои проблемы — это ее проблемы .  . твоя истерика — тоже ее проблема .  . она должна уметь драться за тебя .  . если красота и спасет мир  то вся надежда только на меня . нарисуем твою жену на холсте . ты нам даешь ее фото - мы рисуем ! закажи своей любимой женщине роскошный портрет  чтобы она увидела  какой красивой ты её видишь ! сегодня дарим  картину к заказу  чтобы порадовать еще и маму  сестру или дочку ! чтобы порадовать любимых  не нужен повод . переходи  что бы рассчитать стоимость за  минуту ! https // vk . me / mnogokartinrf ? ref = vk сильный мужчина никогда не поднимет руку на женщину . он ударит по её кошельку ! арнольд шварцнеггер ( с ). девушки  если вы оказались в черном списке у вашего парня — еще не все потеряно . при переводе денег на его карту можно оставить сообщение !  мужчина молодеет до двадцати семи . потом он чертовски хорошеет ! дуэйн джонсон базу говорит вообще - то личный кaнал админа где он вы

In [213]:
text = re.sub(r'\s\s+', ' ', text)

In [214]:
text = re.sub(r'[https].+[vk]', '', text)

In [215]:
censure = r'\$\s+\w+'
text = re.sub(censure, '', text)

In [217]:
sentences = re.split(r'[\.\?!]', text)
sentences[0:50]

['правила жизни джеймса бонда ',
 ' платит женщина ',
 ' всегда ',
 ' ',
 ' твои проблемы — это ее проблемы ',
 ' ',
 ' твоя истерика — тоже ее проблема ',
 ' ',
 ' она должна уметь драться за тебя ',
 ' ',
 ' если красота и спасет мир то вся надежда только на меня ',
 ' нарисуем твою жену на холсте ',
 ' ты нам даешь ее фото - мы рисуем ',
 ' закажи своей любимой женщине роскошный портрет чтобы она увидела какой красивой ты её видишь ',
 ' сегодня дарим картину к заказу чтобы порадовать еще и маму сестру или дочку ',
 ' чтобы порадовать любимых не нужен повод ',
 ' переходи что бы рассчитать стоимость за минуту ',
 '  ',
 ' me / bitchy_man женщина должна зарабатывать больше чем успевает тратить ее мужчина ',
 ' она так развивается ',
 ' при раскладе « все по честному » « все поровну » женщина деградирует ',
 ' у настоящей женщины попросту нет времени на измену — она занята бизнесом чтобы обеспечивать своего единственного ',
 ' женщины запомните мужчину нужно любить как кота ',
 ' ласк

In [218]:
sentences = [sentence.split() for sentence in sentences if sentence]

['платит', 'женщина']

In [219]:
co_occurrence = defaultdict(lambda: defaultdict(int))

In [220]:
def update_co_occurrence(sentence, window_size=1):
    words = ["<START>"] + sentence + ["<END>"]
    for i, word in enumerate(words):
        for j in range(max(0, i - window_size), min(len(words), i + window_size + 1)):
            if i != j:
                co_occurrence[word][words[j]] += 1

for sentence in sentences:
    update_co_occurrence(sentence)

co_occurrence['absurd']

defaultdict(int, {})

In [221]:
tokens = list(co_occurrence.keys())
tokens[:10]

['<START>',
 'правила',
 'жизни',
 'джеймса',
 'бонда',
 '<END>',
 'платит',
 'женщина',
 'всегда',
 'твои']

In [222]:
co_occurrence_df = pd.DataFrame(co_occurrence).fillna(0).astype(int).reindex(tokens, columns=tokens, fill_value=0)
co_occurrence_df

,<START>,правила,жизни,джеймса,бонда,<END>,платит,женщина,всегда,твои,...,значит,плохо,хотела,улыбаюсь,мило,поступаю,гадко,мальчик,загадка,absurd
<START>,0,1,0,0,0,5,1,0,1,1,...,0,0,0,1,0,0,0,0,0,0
правила,1,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
жизни,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
джеймса,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
бонда,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
поступаю,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
гадко,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
мальчик,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
загадка,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
